# References

* http://peter-hoffmann.com/2020/clients-and-data-access-with-turbodbc-to-azure-synapse-sql-on-demand.html
* https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-storage-files-storage-access-control?tabs=shared-access-signature#supported-storage-authorization-types

# Azure Setup with SQL User (database user with password)

Summary of steps:

**Azure Container UI**
* Go to the storage container
* Go to **shared access signature**
* Select **key 1** and the permission your database user should have on the storage container
* Click **Generate SAS Token and URL**
* Pay attention to the **SAS Token** and **URL value** that will be generated below after your Click

**AZURE Synpase Studio**
* Go to the sql serverless pool sql editor in synapse, select built-in and db master (or other pool if you are using another sql pool)
* Replace **<sas_token>** and **<url_value>** below with the generated values from step aboves

**SQL to create a database user with password in sql pool inside synapse**
```sql
CREATE LOGIN testuser WITH password='xxx';
```

**SQL to give database user access to the storage container**
```sql
DROP CREDENTIAL [<url_value>]
CREATE CREDENTIAL [<url_value>]
WITH IDENTITY='SHARED ACCESS SIGNATURE'
, SECRET = '<sas_token>';
GO
USE master
GRANT REFERENCES ON CREDENTIAL::[<url_value>] TO [analyst4];
```



# Connecting to the SQL Pool using Pydobc

You can use the standard SQL Server ODBC driver for connecting to the Synapse SQL pool.

In [1]:
import pyodbc

server ='dyvenia-velux-ondemand.sql.azuresynapse.net'
port = 1433
database="acdb"
uid="analyst4"
pwd="12!jd1kdd303_"
#pwd="Toto!23!45672"

conn = pyodbc.connect(driver= 'ODBC Driver 17 for SQL Server', server=server, user=uid, password=pwd, database=database)

cursor = conn.cursor()

# Running SQL Code on CSV or PARQUET

In [5]:
sql = """select *
from openrowset(
    bulk 'https://dyvenia.blob.core.windows.net/testing/test/data.csv',
    format = 'CSV',
    parser_version = '2.0') as rows
    """
cursor.execute(sql)
cursor.fetchall()

[('country', ' sales'),
 ('italy', ' 100'),
 ('germany', ' 200'),
 ('spain', ' 50')]

# Creating an External Table to Query via SQL from Python

### Create a CREDENTIAL to allow access to the Storage Container

```SQL
USE [acdb]
CREATE DATABASE SCOPED CREDENTIAL [<credential_name>]
WITH IDENTITY='SHARED ACCESS SIGNATURE',  
SECRET = '<Blob SAS token>';
```
### Create a DATA SOURCE using the above credential

```sql
USE [acdb]
CREATE EXTERNAL DATA SOURCE SqlOnDemandDemo WITH (
    LOCATION = 'https://<storage_name>.blob.core.windows.net/<container_name>',
    CREDENTIAL = <credential_name>
);
```

### Create File Formats

Here we create 2: one for csv, one for parquet.

**CSV**

```sql
USE [acdb]
CREATE EXTERNAL FILE FORMAT QuotedCsvWithHeaderFormat
WITH (  
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS ( FIELD_TERMINATOR = ',', STRING_DELIMITER = '"', FIRST_ROW = 1   )
);
```

**PARQUET**

```sql
USE [acdb]
CREATE EXTERNAL FILE FORMAT parquet
WITH (  
    FORMAT_TYPE = PARQUET
);
```

### Create External Table

Creating external table `sales` on `test/data.csv` in container `testing` (the container `testing` is defined in the above sql when creating the `DATA SOURCE`).

**CSV Format**

```sql
USE [acdb];
GO
CREATE EXTERNAL TABLE sales
(
    [country] VARCHAR (100),
    [sales] FLOAT
)
WITH (
    LOCATION = 'test/data.csv',
    DATA_SOURCE = SqlOnDemandDemo,
    FILE_FORMAT = QuotedCsvWithHeaderFormat
);
```

# Using the External Table

After creating the external table `sales` we can use it from Python running standard sql on it. Note how we can create a `calculated field` now as if we were using normal sql code.

In [5]:
sql = """select country, sales, sales * 2 as two_sales from sales"""
cursor.execute(sql)
cursor.fetchall()

[('italy', 100.0, 200.0), ('germany', 200.0, 400.0), ('spain', 50.0, 100.0)]